In [1]:
import d2lzh as d2l
import mxnet as mx
from mxnet import nd

In [21]:
def run(x):
    return [nd.dot(x, x) for _ in range(10)]

In [22]:
x_cpu = nd.random.uniform(shape=(20, 20))
x_gpu = nd.random.uniform(shape=(20, 20), ctx=mx.gpu(0))

In [23]:
run(x_cpu)  # 预热开始
run(x_gpu)
nd.waitall()  # 预热结束

with d2l.Benchmark('Run on CPU.'):
    run(x_cpu)
    nd.waitall()

with d2l.Benchmark('Then run on GPU.'):
    run(x_gpu)
    nd.waitall()

Run. time: 0.0006 sec
Run. time: 0.0000 sec


In [5]:
with d2l.Benchmark('Run on both CPU and GPU in parallel.'):
    run(x_cpu)
    run(x_gpu)
    nd.waitall()

Run on both CPU and GPU in parallel. time: 1.7660 sec


In [6]:
def copy_to_cpu(x):
    return [y.copyto(mx.cpu()) for y in x]

with d2l.Benchmark('Run on GPU.'):
    y = run(x_gpu)
    nd.waitall()

with d2l.Benchmark('Then copy to CPU.'):
    copy_to_cpu(y)
    nd.waitall()
    
# 在计算y[i]的时候系统可以复制y[i-1]，从而减少计算和通信的总运行时间
with d2l.Benchmark('Run and copy in parallel.'):
    y = run(x_gpu)
    copy_to_cpu(y)
    nd.waitall()

Run on GPU. time: 1.4757 sec
Then copy to CPU. time: 0.6981 sec
Run and copy in parallel. time: 1.4794 sec



[[0.5488135  0.5928446  0.71518934 ... 0.5879671  0.67714113 0.89795595]
 [0.59288025 0.92027175 0.01006369 ... 0.44789353 0.46923852 0.9085955 ]
 [0.09617226 0.29361415 0.90337014 ... 0.613934   0.94202113 0.9734874 ]
 ...
 [0.56042254 0.2664899  0.20471796 ... 0.9226652  0.79347354 0.6272908 ]
 [0.4516154  0.7267139  0.16494884 ... 0.03939442 0.8712849  0.07002813]
 [0.78816986 0.40903556 0.5400793  ... 0.52247906 0.6057777  0.4964771 ]]
<NDArray 2000x2000 @cpu(0)>